In [118]:
import numpy as np
import pandas as pd
import pickle
import utils
import time
import tensorflow as tf

In [3]:
emoji_vectors = pickle.load(open('data/emoji_vectors.p', 'rb'))

In [12]:
w2v_vectors = pickle.load(open('data/w2v_vectors.p', 'rb'))

In [5]:
list(emoji_vectors.keys())[0:10]

['🙅', '💂', '🏃', '🤸', '🏌', '0', '👨', '👏', '🚶', '🦑']

In [6]:
list(w2v_vectors.keys())[0:10]

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said']

In [20]:
emoji_embedding = np.array([v for v in emoji_vectors.values()])

In [21]:
emoji_embedding.shape

(1126, 300)

In [13]:
utils.cosine_similarity(emoji_vectors['⭐'], w2v_vectors['star'])

1.0

In [22]:
sess = tf.InteractiveSession()

In [78]:
x = tf.nn.l2_normalize(tf.constant([[1,2,3], [1,2,3]], dtype=tf.float32), axis=1)
y = tf.nn.l2_normalize(tf.constant([[1,2,3], [3,2,1]], dtype=tf.float32), axis=1)
c = tf.losses.cosine_distance(x, y, axis=1, reduction=tf.losses.Reduction.NONE)

sess.run([x,y,c])

[array([[0.26726124, 0.5345225 , 0.8017837 ],
        [0.26726124, 0.5345225 , 0.8017837 ]], dtype=float32),
 array([[0.26726124, 0.5345225 , 0.8017837 ],
        [0.8017837 , 0.5345225 , 0.26726124]], dtype=float32),
 array([[1.1920929e-07],
        [2.8571433e-01]], dtype=float32)]

In [42]:
def matmul3d(X, W):
    """Wrapper for tf.matmul to handle a 3D input tensor X.
    Will perform multiplication along the last dimension.

    Args:
      X: [m,n,k]
      W: [k,l]

    Returns:
      XW: [m,n,l]
    """
    Xr = tf.reshape(X, [-1, tf.shape(X)[2]])
    XWr = tf.matmul(Xr, W)
    newshape = [tf.shape(X)[0], tf.shape(X)[1], tf.shape(W)[1]]
    return tf.reshape(XWr, newshape)

In [108]:
tf.reset_default_graph()
sess.close()
sess = tf.InteractiveSession()

/home/davidhou8791/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [109]:
batch_size = 128
nodes = 300
embed_size = 300
x_seq_length = 10

inputs = tf.placeholder(tf.float32, (None, x_seq_length, embed_size), 'inputs')
input_mean = tf.nn.l2_normalize(tf.reduce_mean(inputs, axis=1), axis=1)

output_embedding = tf.constant(emoji_embedding, name='output_embedding')

lstm_cell = tf.contrib.rnn.LSTMCell(nodes)
lstm_outputs, _ = tf.nn.dynamic_rnn(lstm_cell, inputs=inputs, dtype=tf.float32)
      

logits = tf.layers.dense(lstm_outputs, units=len(emoji_vectors), activation='softmax') 
outputs = matmul3d(logits, output_embedding)

output_mean = tf.nn.l2_normalize(tf.reduce_mean(outputs, axis=1), axis=1)

with tf.name_scope("optimization"):
    # Loss function
    loss = tf.losses.cosine_distance(input_mean, output_mean, axis=1)
    # Optimizer
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [110]:
X = np.array([[w2v_vectors['star']]*x_seq_length for _ in range(batch_size)])
X.shape

(128, 10, 300)

In [123]:
sess.run(tf.global_variables_initializer())
epochs = 50
for i in range(epochs):
    start_time = time.time()
    _, l = sess.run([optimizer, loss], feed_dict={inputs:X})
    if not i%10:
        print('Epoch {:3} Loss: {:>6.3f} Epoch duration: {:>6.3f}s'.format(i, l, time.time() - start_time))

Epoch   0 Loss:  0.757 Epoch duration:  0.666s
Epoch  10 Loss:  0.397 Epoch duration:  0.643s
Epoch  20 Loss:  0.002 Epoch duration:  0.645s
Epoch  30 Loss:  0.001 Epoch duration:  0.642s
Epoch  40 Loss:  0.000 Epoch duration:  0.628s
Epoch  50 Loss:  0.000 Epoch duration:  0.622s


KeyboardInterrupt: 

In [127]:
lo = sess.run(logits, feed_dict={inputs: np.array([w2v_vectors['star']]*x_seq_length).reshape(-1, 10, 300)})

In [130]:
lo.shape

(1, 10, 1126)

In [131]:
np.argmax(lo, axis=2)

array([[984, 984, 984, 984, 984, 984, 984, 984, 984, 984]])

In [133]:
list(emoji_vectors.keys())[984]

'⭐'